# Testing Model
Let's test the model performance on test data

In [1]:
using Flux
using Flux:onehot,crossentropy,onecold
using CSV,MLBase
using DataFrames
using BSON
using Embeddings
using WordTokenizers
using Random

┌ Info: CUDAdrv.jl failed to initialize, GPU functionality unavailable (set JULIA_CUDA_SILENT or JULIA_CUDA_VERBOSE to silence or expand this message)
└ @ CUDAdrv /home/blackforest/.julia/packages/CUDAdrv/mCr0O/src/CUDAdrv.jl:69


In [2]:
emoji_dictionary = Dict{Int64,String}(0=>"💙",    # :heart: prints a black instead of red heart depending on the font
                    1=> "🎾",
                    2=> "😄",
                    3=> "😞",
                    4=> "🍴")

Dict{Int64,String} with 5 entries:
  0 => "💙"
  4 => "🍴"
  2 => "😄"
  3 => "😞"
  1 => "🎾"

In [3]:
const embtable = load_embeddings(GloVe) # or load_embeddings(FastText_Text) or ...

const get_word_index = Dict(word=>ii for (ii,word) in enumerate(embtable.vocab))

function get_embedding(word)
    ind = get_word_index[word]
    emb = embtable.embeddings[:,ind]
    return emb
end

set_tokenizer(poormans_tokenize)

tokenize (generic function with 1 method)

In [4]:
function tokenise(s)
    token_arr = []
    for c in s
        if (c in keys(get_word_index))==1
        push!(token_arr,get_embedding(c))
    else
        #print(c)
        push!(token_arr,get_embedding("unk"))
    
        end
    end
    return token_arr
end

tokenise (generic function with 1 method)

In [6]:
test = CSV.read("./Downloads/emojify/test_emoji.csv",header =["Text","Classifier","Col3","Col4"])[:,[1,2]];
first(test,6)

thread = 1 warning: only found 2 / 4 columns on data row: 1. Filling remaining columns with `missing`
thread = 1 warning: only found 2 / 4 columns on data row: 2. Filling remaining columns with `missing`
thread = 1 warning: only found 2 / 4 columns on data row: 3. Filling remaining columns with `missing`
thread = 1 warning: only found 2 / 4 columns on data row: 4. Filling remaining columns with `missing`
thread = 1 warning: only found 2 / 4 columns on data row: 5. Filling remaining columns with `missing`
thread = 1 warning: only found 2 / 4 columns on data row: 6. Filling remaining columns with `missing`
thread = 1 warning: only found 2 / 4 columns on data row: 7. Filling remaining columns with `missing`
thread = 1 warning: only found 2 / 4 columns on data row: 8. Filling remaining columns with `missing`
thread = 1 warning: only found 2 / 4 columns on data row: 9. Filling remaining columns with `missing`
thread = 1 warning: only found 2 / 4 columns on data row: 10. Filling remaining co

,Text,Classifier
,String,Int64
1,I want to eat\t,4
2,he did not answer\t,3
3,he got a very nice raise\t,2
4,she got me a nice present\t,2
5,ha ha ha it was so funny\t,2
6,he is a good friend\t,2


Now, Let's load the data

In [67]:
BSON.@load "./Downloads/emojify/emojifier_norm.bson" Scanner max_length

Before testing the data, we first need to pre-process the data same as we did while training

In [68]:
X_test = test[:,1]
Y_test = test[:,2]
N = 5
Y_ = zeros(N,length(Y_test))
for i in 1:length(Y_test)
    Y_[Y_test[i]+1,i] = 1
end

X_t = [tokenize(lowercase(i)) for i in X_test] 
Xs_t = [tokenise(a) for a in X_t];

X1_t = []
for i in 1:length(Xs_t)
    m = fill(0.0,(length(Xs_t[1][1]),max_length))
    for j =1:length(Xs_t[i])
        for k in 1:length(Xs_t[i][j])
            m[k,j] = Xs_t[i][j][k]
        end
    end
    push!(X1_t,m)
end

Y1_t = []
for i in 1:size(Y_)[2]
    push!(Y1_t,Y_[:,i])
end

for i in 1:length(X_test)
    X1_t[i] = Flux.normalise(X1_t[i],dims = 2)
end

In [69]:
#Testing the Label
test_rand = (abs.(rand(Int,5).%56)).+1
y_label = []
for i in 1:length(X1_t)
    push!(y_label,Scanner(X1_t[i]))
    Flux.reset!(Scanner)
end
print("Example: ")
for i in test_rand
    print("\nText: ",X_test[i],"\n","Real Emoji: ",emoji_dictionary[Y_test[i]])
    print("\nPredicted: ", emoji_dictionary[argmax(y_label[i][:,max_length])-1],"\n\n")
end

Example: 
Text: I am upset	
Real Emoji: 😞
Predicted: 😞


Text: she is a bully	
Real Emoji: 😞
Predicted: 💙


Text: he is a good friend	
Real Emoji: 😄
Predicted: 😄


Text: yesterday we lost again	
Real Emoji: 😞
Predicted: 🎾


Text: will you be my valentine	
Real Emoji: 😄
Predicted: 😞



In [70]:
y_pred = []
for i in 1:length(y_label)
    push!(y_pred,Int64(argmax(y_label[i][:,max_length])-1))
    end
y_pred = Int64.(y_pred);

confusmat(5,Y_test.+1,y_pred.+1)

5×5 Array{Int64,2}:
 6  1   0  0  0
 0  6   0  1  1
 5  1  10  2  0
 5  1   1  8  1
 0  0   3  0  4

In [71]:
function accuracy(x,y)
    y_hat = []
    for i in 1:length(x)
        push!(y_hat,Scanner(x[i]))
        Flux.reset!(Scanner)
    end
    sum=0.0
    for i in 1:length(y)
        if (argmax(y_hat[i][:,max_length])==argmax(y[i]))
        sum+=1.0
        end
    end
    return sum/length(y)
end

accuracy (generic function with 1 method)

In [72]:
accuracy(X1_t,Y1_t)

0.6071428571428571